# Pystac Tutorial
## Create and manipulate Spacenet Rio STAC

This tutorial shows how to create and manipulate STACs using pystac. It contains two parts:

1. Create Spacenet Rio STAC
    - Create (in memory) a pystac catalog of [Spacenet 1 imagery from the Rio De Janeiro AOI](https://spacenet.ai/rio-de-janeiro/) using data hosted in a public s3 bucket
    - Set uri's based on a root directory
    - Save the STAC locally
    
2. Create a new STAC with COGs and labels
    - Load the STAC that was created and saved in step 1
    - Determine which images included in the STAC overlap with building labels
    - For has labeled buildings, create and save off both COGs and geojsons of building labels
    - Create an updated STAC that points to the new files and ony includes labeled scenes
    - Set uri's based on a root directory
    - Save the STAC locally

In [2]:
import sys
sys.path.append('..')

In [7]:
from datetime import datetime
from os.path import basename, join
from subprocess import call

import boto3
import geopandas as gpd
import rasterio
from pystac import (Catalog, Collection, Extent, Item, SpatialExtent,
                    TemporalExtent)
from shapely.geometry import Polygon, box

### Create Spacenet Rio STAC

In [ ]:
# Initialize a Spacenet 1 STAC
spacenet = Catalog(id='spacenet', description='Spacenet 1 STAC')

In [ ]:
# Create a spatial extent object using the AOI geojson on S3
aoi = 's3://spacenet-dataset/AOIs/AOI_1_Rio/metadata/AOI_1_Rio_metadata_public_AOI_outline.geojson'
sp_extent = SpatialExtent(list(gpd.read_file(aoi)['geometry'][0].bounds))

In [ ]:
# The capture date for Spacenet 1 Rio imagery is unspecified in the dataset page so we will use 
# October 22, 2015 as an example date.

# Create a python datetime object
capture_date = datetime.strptime('2015-10-22', '%Y-%m-%d') 
tmp_extent = TemporalExtent([(capture_date, None)])

In [ ]:
# Create an Extent object that will define both the spatial and temporal extents of the Rio collection
extent = Extent(sp_extent, tmp_extent)

In [10]:
rio = Collection(id='rio', description = 'Rio Spacenet 3 dataset', extent = extent)
spacenet.add_child(rio)
client = boto3.client('s3')
x = client.list_objects(Bucket='spacenet-dataset', Prefix='AOIs/AOI_1_Rio/srcData/mosaic_3band/')['Contents']

for scene in x:
    uri = join('s3://spacenet-dataset/', scene['Key'])
    params = {}
    params['id'] = basename(uri).split('.')[0]
    with rasterio.open(uri) as src:
        params['bbox'] = list(src.bounds)
        params['geometry'] = box(*params['bbox']).__geo_interface__
    params['datetime'] = capture_date
    params['properties'] = {}
    i = Item(**params)
    i.add_asset(key='image', href=uri, title='Geotiff', media_type='image/tiff')
    rio.add_item(i)

In [ ]:
spacenet_cog.set_uris_from_root('spacenet-stac/')
spacenet_cog.save()

### Create new STAC with COGs and labels

In [11]:
labels = gpd.read_file('s3://spacenet-dataset/AOIs/AOI_1_Rio/geojson_buildings/AOI_1_Rio_geojson_buildings.geojson')

In [12]:
def cogify_if_labeled(item, labels, data_dir):
    scene_labels = labels[labels.within(Polygon(item.geometry['coordinates'][0]))]
    if len(scene_labels) == 0:
        item.id += '-no-labels'
    else:
        label_uri = join(data_dir, '{}-labels.geojson'.format(item.id))
        scene_labels.to_file(label_uri, driver='GeoJSON')
        item.add_asset(key='label', href=label_uri, media_type='application/geo+json')
        output_cog_uri = join(data_dir, '{}-cog.tif'.format(item.id))
        call(' '.join(['rio', 'cogeo', 'create', item.assets['image'].href, output_cog_uri]), shell=True)
        item.assets['image'].href = output_cog_uri
    return item

In [13]:
spacenet_cog = spacenet.map_items(cogify_if_labeled, labels=labels, data_dir='/home/simon/Desktop/')

In [27]:
new_links = []
for link in spacenet_cog.get_child('rio').links:
    if link.rel != 'item':
        new_links.append(link)
    elif not link.target.id.endswith('00'):
        new_links.append(link)

In [29]:
spacenet_cog.get_child('rio').links = new_links

In [34]:
spacenet_cog.set_uris_from_root('spacenet-cog-stac/')
spacenet_cog.save()